### 출처: Won Joon Yoo, Introduction to Deep Learning for Natural Language Processing, Wikidocs</br>
### https://wikidocs.net/book/2155

### 글자 단위 RNN(Char RNN)으로 텍스트 생성하기

#### 다 대 일 예측

In [2]:
import numpy as np
import tensorflow
from tensorflow.keras.utils import to_categorical

In [3]:
text='''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''


In [4]:
# 단락 구분을 없애고 하나의 문자열로 저장
tokens = text.split() # '\n 제거'
text = ' '.join(tokens)
print(text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


In [5]:
char_vocab = sorted(list(set(text))) # 중복을 제거한 글자 집합 생성
print(char_vocab)

[' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [6]:
vocab_size=len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 33


In [7]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


### Example) 5개의 입력 글자 시퀀스로부터 다음 글자 시퀀스를 예측. 즉, RNN의 time step은 5번
### 다대일 예측
stude -> n  
tuden -> t

In [8]:
# 입력 시퀀스의 길이를 10으로 임의 설정. 예측 대상 글자 길이 11
length = 11
sequences = []
for i in range(length, len(text)):
    seq = text[i-length:i] # 길이 11의 문자열을 지속적으로 만든다.
    sequences.append(seq)
print('총 훈련 샘플의 수: %d' % len(sequences))

총 훈련 샘플의 수: 426


In [9]:
sequences[:10]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a']

In [10]:
X = []
for line in sequences: # 전체 데이터에서 문장 샘플을 1개씩 꺼낸다.
    temp_X = [char_to_index[char] for char in line] # 문장 샘플에서 각 글자에 대해서 정수 인코딩을 수행.
    X.append(temp_X)

In [11]:
for line in X[:5]:
    print(line)

[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18]
[0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28]
[16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17]
[14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0]
[28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]


In [12]:
# feature와 label 분리
sequences = np.array(X)
X = sequences[:,:-1]
y = sequences[:,-1] # 맨 마지막 위치의 글자를 분리

In [13]:
for line in X[:5]:
    print(line)

[ 8  0 16 14 28  0 24 23  0 31]
[ 0 16 14 28  0 24 23  0 31 18]
[16 14 28  0 24 23  0 31 18 28]
[14 28  0 24 23  0 31 18 28 17]
[28  0 24 23  0 31 18 28 17  0]


In [14]:
print(y[:5])

[18 28 17  0 21]


In [15]:
# 원-핫 인코딩
sequences = [to_categorical(x, num_classes=vocab_size) for x in X] # X에 대한 원-핫 인코딩
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size) # y에 대한 원-핫 인코딩

In [16]:
# (a, b, c)
# a = 샘플의 수 = No. of samples = batch_size
# b = 입력 시퀀스의 길이 = input_length = timeseries = timesteps
# c = 각 벡터의 차원 = input dim = Dimensionality of word representation
print(X.shape)

(426, 10, 33)


### 모델 설계

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
model = Sequential()
model.add(LSTM(80, input_shape=(X.shape[1], X.shape[2]))) # X.shape[1]은 10, X.shape[2]는 33
# 출력층에 단어 집합의 크기만큼의 뉴런을 배치
model.add(Dense(vocab_size, activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
14/14 - 0s - loss: 3.4678 - accuracy: 0.1432
Epoch 2/100
14/14 - 0s - loss: 3.3298 - accuracy: 0.1972
Epoch 3/100
14/14 - 0s - loss: 3.0678 - accuracy: 0.1972
Epoch 4/100
14/14 - 0s - loss: 2.9806 - accuracy: 0.1972
Epoch 5/100
14/14 - 0s - loss: 2.9558 - accuracy: 0.1972
Epoch 6/100
14/14 - 0s - loss: 2.9387 - accuracy: 0.1972
Epoch 7/100
14/14 - 0s - loss: 2.9263 - accuracy: 0.1972
Epoch 8/100
14/14 - 0s - loss: 2.8956 - accuracy: 0.1972
Epoch 9/100
14/14 - 0s - loss: 2.8689 - accuracy: 0.1972
Epoch 10/100
14/14 - 0s - loss: 2.8551 - accuracy: 0.1972
Epoch 11/100
14/14 - 0s - loss: 2.8155 - accuracy: 0.1972
Epoch 12/100
14/14 - 0s - loss: 2.7789 - accuracy: 0.1972
Epoch 13/100
14/14 - 0s - loss: 2.7324 - accuracy: 0.1995
Epoch 14/100
14/14 - 0s - loss: 2.6813 - accuracy: 0.2066
Epoch 15/100
14/14 - 0s - loss: 2.6215 - accuracy: 0.2559
Epoch 16/100
14/14 - 0s - loss: 2.5835 - accuracy: 0.2582
Epoch 17/100
14/14 - 0s - loss: 2.5295 - accuracy: 0.2418
Epoch 18/100
14/14 - 0s

In [21]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):
# 모델, 인덱스 정보, 문장 길이, 초기 시퀀스, 반복 횟수
    init_text = seed_text # 문장 생성에 사용할 초기 시퀀스
    sentence = ''

    for _ in range(n): # n번 반복
        encoded = [char_to_index[char] for char in seed_text] # 현재 시퀀스에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre') # 데이터에 대한 패딩
        encoded = to_categorical(encoded, num_classes=len(char_to_index))
        result = model.predict_classes(encoded, verbose=0)
        # 입력한 X(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 글자)를 result에 저장.
        for char, index in char_to_index.items(): # 만약 예측한 글자와 인덱스와 동일한 글자가 있다면
            if index == result: # 해당 글자가 예측 글자이므로 break
                break
        seed_text=seed_text + char # 현재 시퀀스 + 예측 글자를 현재 시퀀스로 변경
        sentence=sentence + char # 예측 글자를 문장에 저장
        # for문이므로 이 작업을 다시 반복

    sentence = init_text + sentence
    return sentence

In [22]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
I get on with life as a programmer, I like to use words about beer. But when I stapt to ag
